In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_excel('../customer_call/customer-call-list.xlsx')

df.head(3)

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact,Not_Useful_Column
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Yes,No,True
1,1002,Abed,Nadir,123/643/9775,93 West Main Street,No,Yes,False
2,1003,Walter,/White,7066950392,298 Drugs Driveway,N,NaN,True


In [3]:
#Copy of dataframe
df_copy = df.copy()

In [4]:
#DF dimension
df_copy.shape

(21, 8)

In [5]:
#Remove unwanted cols
df_copy = df_copy.drop(['Not_Useful_Column'], axis=1) # axis=1 means column-wise drop, axis=0 means row-wise drop

In [6]:
# df_copy.shape
df_copy.head(3)

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Yes,No
1,1002,Abed,Nadir,123/643/9775,93 West Main Street,No,Yes
2,1003,Walter,/White,7066950392,298 Drugs Driveway,N,NaN


In [7]:
#Drop duplicates
df_copy = df_copy.drop_duplicates()

df_copy.shape

(20, 7)

copy of df
fix col headers - df_copy.columns
remove unwanted col
drop dups
standardize the data - remove whitespaces, standardize phone_num
split "Address" col into multiple cols

In [8]:
df_copy

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Yes,No
1,1002,Abed,Nadir,123/643/9775,93 West Main Street,No,Yes
2,1003,Walter,/White,7066950392,298 Drugs Driveway,N,NaN
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",Yes,Y
4,1005,Jon,Snow,876|678|3469,123 Dragons Road,Y,No
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,Yes,Yes
6,1007,Jeff,Winger,NaN,1209 South Street,No,No
7,1008,Sherlock,Holmes,876|678|3469,98 Clue Drive,N,No
8,1009,Gandalf,NaN,N/a,123 Middle Earth,Yes,NaN
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",Yes,No


In [9]:
#Remove leading + trailing whitespaces
df_copy['Last_Name'] = df_copy['Last_Name'].str.strip('./_')

df_copy

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Yes,No
1,1002,Abed,Nadir,123/643/9775,93 West Main Street,No,Yes
2,1003,Walter,White,7066950392,298 Drugs Driveway,N,NaN
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",Yes,Y
4,1005,Jon,Snow,876|678|3469,123 Dragons Road,Y,No
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,Yes,Yes
6,1007,Jeff,Winger,NaN,1209 South Street,No,No
7,1008,Sherlock,Holmes,876|678|3469,98 Clue Drive,N,No
8,1009,Gandalf,NaN,N/a,123 Middle Earth,Yes,NaN
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",Yes,No


In [10]:
#Standardize col "phone_number" as follows: 123-545-5421

def clean_phone_number(phone_number):
    """Cleans and standardizes phone numbers.

    Args:
        phone_number: The phone number to clean.

    Returns:
        The cleaned phone number in the format "XXX-XXX-XXXX", or None if the
        input is missing or invalid.
    """
    if pd.isna(phone_number) or str(phone_number).lower() in ("n/a", "na"):
        return None

    # Remove non-numeric characters and plus signs
    digits = ''.join(c for c in str(phone_number) if c.isdigit() or c == '+')

    # Ensure we have at least 10 digits (assuming North American Numbering Plan)
    if len(digits) < 10:
        return None

    # Handle phone numbers with leading plus sign
    if digits.startswith('+'):
        return f"{digits[1:4]}-{digits[4:7]}-{digits[7:]}"
    else:
        return f"{digits[:3]}-{digits[3:6]}-{digits[6:]}"

# Clean phone numbers
df_copy['Phone_Number'] = df_copy['Phone_Number'].apply(clean_phone_number)

df_copy

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Yes,No
1,1002,Abed,Nadir,123-643-9775,93 West Main Street,No,Yes
2,1003,Walter,White,706-695-0392,298 Drugs Driveway,N,NaN
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",Yes,Y
4,1005,Jon,Snow,876-678-3469,123 Dragons Road,Y,No
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,Yes,Yes
6,1007,Jeff,Winger,None,1209 South Street,No,No
7,1008,Sherlock,Holmes,876-678-3469,98 Clue Drive,N,No
8,1009,Gandalf,NaN,None,123 Middle Earth,Yes,NaN
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",Yes,No


In [11]:
# Split the 'Address' column into multiple columns
new_columns = ['Street_Address', 'State', 'Zip_Code']  # Customize column names as needed
df_copy[new_columns] = df_copy['Address'].str.split(',', expand=True)

df_copy

,CustomerID,First_Name,Last_Name,Phone_Number,Address,Paying Customer,Do_Not_Contact,Street_Address,State,Zip_Code
0,1001,Frodo,Baggins,123-545-5421,"123 Shire Lane, Shire",Yes,No,123 Shire Lane,Shire,None
1,1002,Abed,Nadir,123-643-9775,93 West Main Street,No,Yes,93 West Main Street,None,None
2,1003,Walter,White,706-695-0392,298 Drugs Driveway,N,NaN,298 Drugs Driveway,None,None
3,1004,Dwight,Schrute,123-543-2345,"980 Paper Avenue, Pennsylvania, 18503",Yes,Y,980 Paper Avenue,Pennsylvania,18503
4,1005,Jon,Snow,876-678-3469,123 Dragons Road,Y,No,123 Dragons Road,None,None
5,1006,Ron,Swanson,304-762-2467,768 City Parkway,Yes,Yes,768 City Parkway,None,None
6,1007,Jeff,Winger,None,1209 South Street,No,No,1209 South Street,None,None
7,1008,Sherlock,Holmes,876-678-3469,98 Clue Drive,N,No,98 Clue Drive,None,None
8,1009,Gandalf,NaN,None,123 Middle Earth,Yes,NaN,123 Middle Earth,None,None
9,1010,Peter,Parker,123-545-5421,"25th Main Street, New York",Yes,No,25th Main Street,New York,None


In [12]:
#Drop col "Address"
df_copy = df_copy.drop('Address', axis=1)

df_copy

,CustomerID,First_Name,Last_Name,Phone_Number,Paying Customer,Do_Not_Contact,Street_Address,State,Zip_Code
0,1001,Frodo,Baggins,123-545-5421,Yes,No,123 Shire Lane,Shire,None
1,1002,Abed,Nadir,123-643-9775,No,Yes,93 West Main Street,None,None
2,1003,Walter,White,706-695-0392,N,NaN,298 Drugs Driveway,None,None
3,1004,Dwight,Schrute,123-543-2345,Yes,Y,980 Paper Avenue,Pennsylvania,18503
4,1005,Jon,Snow,876-678-3469,Y,No,123 Dragons Road,None,None
5,1006,Ron,Swanson,304-762-2467,Yes,Yes,768 City Parkway,None,None
6,1007,Jeff,Winger,None,No,No,1209 South Street,None,None
7,1008,Sherlock,Holmes,876-678-3469,N,No,98 Clue Drive,None,None
8,1009,Gandalf,NaN,None,Yes,NaN,123 Middle Earth,None,None
9,1010,Peter,Parker,123-545-5421,Yes,No,25th Main Street,New York,None


In [15]:
#Standardize col "Paying Customer"
df_copy['Paying Customer'] = df_copy['Paying Customer'].str.replace('Yes', 'Y',).str.replace('No', 'N') 

df_copy

,CustomerID,First_Name,Last_Name,Phone_Number,Paying Customer,Do_Not_Contact,Street_Address,State,Zip_Code
0,1001,Frodo,Baggins,123-545-5421,Y,No,123 Shire Lane,Shire,None
1,1002,Abed,Nadir,123-643-9775,N,Yes,93 West Main Street,None,None
2,1003,Walter,White,706-695-0392,N,NaN,298 Drugs Driveway,None,None
3,1004,Dwight,Schrute,123-543-2345,Y,Y,980 Paper Avenue,Pennsylvania,18503
4,1005,Jon,Snow,876-678-3469,Y,No,123 Dragons Road,None,None
5,1006,Ron,Swanson,304-762-2467,Y,Yes,768 City Parkway,None,None
6,1007,Jeff,Winger,None,N,No,1209 South Street,None,None
7,1008,Sherlock,Holmes,876-678-3469,N,No,98 Clue Drive,None,None
8,1009,Gandalf,NaN,None,Y,NaN,123 Middle Earth,None,None
9,1010,Peter,Parker,123-545-5421,Y,No,25th Main Street,New York,None


In [16]:
#Standardize col "Do_Not_Contact"
df_copy['Do_Not_Contact'] = df_copy['Do_Not_Contact'].str.replace('Yes', 'Y',).str.replace('No', 'N') 

df_copy

,CustomerID,First_Name,Last_Name,Phone_Number,Paying Customer,Do_Not_Contact,Street_Address,State,Zip_Code
0,1001,Frodo,Baggins,123-545-5421,Y,N,123 Shire Lane,Shire,None
1,1002,Abed,Nadir,123-643-9775,N,Y,93 West Main Street,None,None
2,1003,Walter,White,706-695-0392,N,NaN,298 Drugs Driveway,None,None
3,1004,Dwight,Schrute,123-543-2345,Y,Y,980 Paper Avenue,Pennsylvania,18503
4,1005,Jon,Snow,876-678-3469,Y,N,123 Dragons Road,None,None
5,1006,Ron,Swanson,304-762-2467,Y,Y,768 City Parkway,None,None
6,1007,Jeff,Winger,None,N,N,1209 South Street,None,None
7,1008,Sherlock,Holmes,876-678-3469,N,N,98 Clue Drive,None,None
8,1009,Gandalf,NaN,None,Y,NaN,123 Middle Earth,None,None
9,1010,Peter,Parker,123-545-5421,Y,N,25th Main Street,New York,None


In [17]:
df_copy = df_copy[['Paying Customer', 'Do_Not_Contact']].fillna('', inplace=True)

df_copy

/var/folders/40/bphxdxh14n95ft8310ld7dn40000gn/T/ipykernel_76807/2073174408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy = df_copy[['Paying Customer', 'Do_Not_Contact']].fillna('', inplace=True)
